# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  *   Emmanuel Merida Toledo **A01795858**
  *   Carlos Pano Hernandez **A01066264**
  *   Mireya Isabel Pérez del Razo **A01795608**
  *   Scarlet Marie Wessin Romero **A01796242**

* **Número de Equipo:**
  * 64

* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

## **Introducción y justificación de la problemática a resolver**

I. Introducción: La medicina preventiva como parte importante de la salud pública, es uno de los temas de interés de la Inteligencia Artificial. Y es uno de los que suelen atenderse a través de chatbots; es decir, a través de programas que permiten simular una conversación con personas, generalmente a través de texto o de voz. En el ámbito médico el uso de chatbots es un tema delicado, considerando que la precisión de la respuesta tendrá repercusión directa sobre la salud del consultante; sin embargo, y como resultado del constante entrenamiento de los algoritmos y de la calidad de la información que forma parte del corpus, se ha logrado alcanzar un grado de madurez que permite ofrecer resultados de excelente credibilidad. En este ejercicio se desarrolló un chatbot con la integración de modelos de OpenAI (LLM) y de un sistema RAG que responde correctamente a las preguntas enviadas sobre dos importantes enfermedades atendidas por la medicina preventiva: la diabetes y la hipertensión.

II. Justificación: Como sabemos, la diabetes y la hipertensión son dos de las enfermedades más comunes a nivel mundial, enfermedades que al coexistir disminuyen notablemente la calidad de vida de las personas que las padecen; por lo tanto, es de suma importancia tratar de prevenirlas. Hoy en día dentro de las actividades preventivas se encuentran programas presenciales de difusión, pero no todas las personas tienen acceso a ellos, sobre todo por la falta de tiempo para asistir a dichos eventos y justamente es ahí donde el desarrollo de nuestro chatbot se justifica. El procesamiento del lenguaje natural ha facilitado la comunicación entre las personas y la Inteligencia Artificial, la confiable respuesta a sus preguntas es casi inmediata. Dicho esto, a través del uso de la Inteligencia Artificial podemos acercar la información preventiva al público en general.



## **Sistema RAG + LLM**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
!pip install faiss-cpu

In [ ]:
import os
import openai
import glob
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# === Leer documentos desde carpeta ./docs/ ===
ruta_docs = "/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 3/Semana 9/9.2 Actividad chatbot  RAG + LLM/docs/*.txt"
corpus = []

for archivo in glob.glob(ruta_docs):
    with open(archivo, "r", encoding="utf-8") as f:
        texto = f.read()
        partes = [texto[i:i+1000] for i in range(0, len(texto), 1000)]
        corpus.extend(partes)

print("Documentos encontrados:")
for archivo in glob.glob(ruta_docs):
    print(archivo)

Documentos encontrados:
/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 3/Semana 9/9.2 Actividad chatbot  RAG + LLM/docs/alimentacion.txt
/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 3/Semana 9/9.2 Actividad chatbot  RAG + LLM/docs/diabetes.txt
/content/drive/MyDrive/MNA/Procesamiento de Lenguaje Natural/Modulo 3/Semana 9/9.2 Actividad chatbot  RAG + LLM/docs/hipertension.txt


In [ ]:
# === Crear embeddings y FAISS ===
if not corpus:
    print("Warning: No text files found in the ./docs/ directory or the files are empty. The corpus is empty.")

if corpus:
    modelo = SentenceTransformer("distiluse-base-multilingual-cased-v1")
    embeddings = modelo.encode(corpus)
    if embeddings.shape[0] > 0:
        index = faiss.IndexFlatL2(embeddings[0].shape[0])
        index.add(embeddings)
    else:
        print("Warning: Embeddings could not be created from the corpus.")
else:
    print("Warning: Skipping embedding and FAISS creation due to empty corpus.")

In [ ]:
def responder_con_rag(pregunta, top_k=3):
    emb_pregunta = modelo.encode([pregunta])
    distancias, indices = index.search(emb_pregunta, top_k)
    contexto = "\n\n".join([corpus[i] for i in indices[0]])

    prompt = f"""
Eres un experto en salud preventiva. Responde en español de forma clara y precisa
basándote en el siguiente contexto. Si la información no es suficiente, indica que no se puede responder con certeza.

Contexto:
{contexto}

Pregunta:
{pregunta}

Respuesta:
"""
    client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

    respuesta = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=700
    )

    return respuesta.choices[0].message.content

## **El chatbot, incluyendo ejemplos de prueba**

In [ ]:
respuesta = responder_con_rag("¿Qué alimentos ayudan a prevenir la diabetes?")
print("\nRespuesta del chatbot:\n", respuesta)


Respuesta del chatbot:
 Para prevenir la diabetes tipo 2, es recomendable adoptar una alimentación equilibrada que incluya frutas, vegetales, legumbres, granos enteros y frutos secos. Además, se debe reducir el consumo de azúcares libres y grasas saturadas. También es aconsejable evitar alimentos altos en azúcar, grasa y sal, así como el exceso de alimentos ultraprocesados como frituras, golosinas y comida rápida.


In [ ]:
respuesta = responder_con_rag("¿Cómo se puede prevenir la hipertensión arterial?")
print("\nRespuesta del chatbot:\n", respuesta)


Respuesta del chatbot:
 La hipertensión arterial se puede prevenir siguiendo varias recomendaciones. La Organización Mundial de la Salud sugiere reducir la ingesta de sal a menos de 5 gramos diarios, mantener una dieta rica en frutas y verduras, realizar actividad física regularmente, mantener un peso saludable, y evitar el consumo de alcohol y tabaco. Además, es esencial el diagnóstico temprano mediante controles periódicos de la presión arterial para prevenir complicaciones graves.


In [ ]:
respuesta = responder_con_rag("¿Qué es una alimentación saludable?")
print("\nRespuesta del chatbot:\n", respuesta)


Respuesta del chatbot:
 Una alimentación saludable es aquella que incluye diariamente frutas y verduras, cereales integrales, legumbres, granos enteros y frutos secos, y evita alimentos altos en azúcar, grasa y sal. Se recomienda tomar agua simple como principal bebida y reducir el consumo de bebidas azucaradas. También se aconseja evitar el exceso de alimentos ultraprocesados como frituras, golosinas y comida rápida. Además, comer en horarios regulares y en porciones adecuadas contribuye al bienestar general. Este tipo de alimentación es clave para prevenir enfermedades como la obesidad, la diabetes y la hipertensión.


In [ ]:
respuesta = responder_con_rag("¿Cuánto ejercicio debo hacer por semana para mantenerme sano?")
print("\nRespuesta del chatbot:\n", respuesta)


Respuesta del chatbot:
 Se recomienda realizar actividad física moderada durante al menos 30 minutos al día, cinco días a la semana.


In [ ]:
respuesta = responder_con_rag("¿Qué hábitos diarios ayudan a prevenir enfermedades crónicas?")
print("\nRespuesta del chatbot:\n", respuesta)


Respuesta del chatbot:
 Los hábitos diarios que ayudan a prevenir enfermedades crónicas incluyen:

1. Mantener una alimentación saludable y equilibrada, rica en frutas, vegetales, legumbres, cereales integrales y frutos secos.
2. Evitar alimentos altos en azúcar, grasa y sal, así como el consumo excesivo de alimentos ultraprocesados como frituras, golosinas y comida rápida.
3. Reducir la ingesta de sal a menos de 5 gramos diarios.
4. Tomar agua simple como principal bebida y reducir el consumo de bebidas azucaradas.
5. Realizar actividad física con regularidad, preferentemente moderada durante al menos 30 minutos al día, cinco días a la semana.
6. Mantener un peso corporal saludable.
7. Evitar el consumo de alcohol y tabaco.
8. Comer en horarios regulares y en porciones adecuadas.
9. Realizar controles periódicos de la presión arterial para un diagnóstico temprano de hipertensión.

Estos hábitos deben ser promovidos desde la infancia para mantener una buena salud a lo largo de la vida

# **Conclusiones:**

* #### **Incluyan sus conclusiones de la actividad chatbot LLM + RAG:**

En esta actividad, se desarrolló un chatbot basado en la metodología de Generación Aumentada por Recuperación (RAG), cuyo objetivo principal fue construir un sistema que pueda responder de forma precisa y fundamentada a consultas relacionadas con salud y bienestar, utilizando un corpus específico de documentos.

Se implementaron varias etapas clave:

- *Preparación del corpus:* Se recopilaron textos sobre prevención de enfermedades, creando una base de conocimiento confiable.
- *Vectorización y búsqueda:* Se empleó un modelo sentence-transformer para transformar tanto el corpus como las preguntas en embeddings. Además, se utilizó un índice FAISS para realizar búsquedas semánticas rápidas y efectivas, recuperando fragmentos relevantes según cada consulta.
- *Generación de respuestas:* Se integró un Modelo de Lenguaje Grande (LLM) a través de la API de OpenAI, que combinó la pregunta del usuario con el contexto recuperado para generar respuestas coherentes, precisas y basadas en información confiable.

Durante el proceso, enfrentamos desafíos técnicos como el error NameError causado por variables no definidas en ciertas partes del código. Esto resaltó la importancia de gestionar cuidadosamente el estado y las dependencias en sistemas complejos. La depuración y la atención a la estructura del código fueron fundamentales para asegurar el correcto funcionamiento del sistema.

En conclusión, la arquitectura RAG demuestra ser una estrategia muy potente para mitigar las limitaciones tradicionales de los modelos de lenguaje, como las alucinaciones o la falta de conocimiento actualizado. Al fundamentar las respuestas en una base de conocimiento concreta, se incrementa significativamente la fiabilidad y precisión del chatbot. Además, la integración de técnicas de recuperación de información y generación de lenguaje natural abre nuevas oportunidades para aplicaciones en dominios especializados donde la exactitud y la confianza en la información son cruciales. Esta experiencia práctica nos permitió entender mejor cómo combinar estas tecnologías para construir sistemas de IA avanzados, con potencial de impacto en diferentes áreas en las que la precisión y fundamentación son prioritarias.

# **Fin de la actividad chatbot: LLM + RAG**